In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
data=pd.read_csv('/content/sample_data/blogs_categories.csv')
data

,Unnamed: 0,Data,Labels
0,0,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49...,alt.atheism
1,1,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
2,2,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
3,3,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
4,4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
...,...,...,...
19992,19992,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,talk.religion.misc
19993,19993,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,talk.religion.misc
19994,19994,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
19995,19995,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19997 entries, 0 to 19996
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  19997 non-null  int64 
 1   Data        19997 non-null  object
 2   Labels      19997 non-null  object
dtypes: int64(1), object(2)
memory usage: 468.8+ KB


In [14]:
data.describe()

,Unnamed: 0
count,19997.000000
mean,9998.000000
std,5772.781002
min,0.000000
25%,4999.000000
50%,9998.000000
75%,14997.000000
max,19996.000000


In [15]:
data.isnull().sum()

Unnamed: 0    0
Data          0
Labels        0
dtype: int64

In [16]:
data.duplicated().sum()

0

In [17]:
data.drop_duplicates(inplace=True)

In [18]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK data files (run this once)
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Function to clean text
def preprocess_text(text):
  text = text.lower()  # Convert to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
  tokens = word_tokenize(text)  # Tokenize the text
  stop_words = set(stopwords.words('english'))  # Load stopwords
  filtered_tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
  return ' '.join(filtered_tokens)

# Apply preprocessing
data['cleaned_text'] = data['Data'].apply(preprocess_text)
data['cleaned_text']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        xref cantaloupesrvcscmuedu altatheism49960 alt...
1        xref cantaloupesrvcscmuedu altatheism51060 alt...
2        newsgroups altatheism path cantaloupesrvcscmue...
3        xref cantaloupesrvcscmuedu altatheism51120 alt...
4        xref cantaloupesrvcscmuedu altatheism51121 soc...
                               ...                        
19992    xref cantaloupesrvcscmuedu altatheism54482 tal...
19993    xref cantaloupesrvcscmuedu altatheism54485 tal...
19994    xref cantaloupesrvcscmuedu talkreligionmisc845...
19995    xref cantaloupesrvcscmuedu talkreligionmisc845...
19996    xref cantaloupesrvcscmuedu talkabortion121820 ...
Name: cleaned_text, Length: 19997, dtype: object

In [19]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
x= tfidf_vectorizer.fit_transform(data['cleaned_text'])

# The labels for classification
y = data['Labels']


In [20]:
data['Labels']

0               alt.atheism
1               alt.atheism
2               alt.atheism
3               alt.atheism
4               alt.atheism
                ...        
19992    talk.religion.misc
19993    talk.religion.misc
19994    talk.religion.misc
19995    talk.religion.misc
19996    talk.religion.misc
Name: Labels, Length: 19997, dtype: object

In [21]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train,y_train

(<15997x258828 sparse matrix of type '<class 'numpy.float64'>'
 	with 2116539 stored elements in Compressed Sparse Row format>,
 5748             comp.windows.x
 1688              comp.graphics
 8174            rec.motorcycles
 10194          rec.sport.hockey
 4667      comp.sys.mac.hardware
                   ...          
 11284                 sci.crypt
 11964                 sci.crypt
 5390             comp.windows.x
 860                 alt.atheism
 15795    soc.religion.christian
 Name: Labels, Length: 15997, dtype: object)

In [22]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(15997, 258828)
(4000, 258828)
(15997,)
(4000,)


In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [24]:
nb=MultinomialNB()
nb.fit(x_train,y_train)

MultinomialNB()

In [25]:
y_pred=nb.predict(x_test)
y_pred

array(['rec.sport.hockey', 'comp.sys.mac.hardware',
       'comp.sys.ibm.pc.hardware', ..., 'rec.motorcycles',
       'talk.politics.guns', 'comp.windows.x'], dtype='<U24')

In [30]:
from sklearn import metrics
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred))
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Naive Bayes model accuracy(in %): 0.89825
Naive Bayes model accuracy(in %): 89.825


In [31]:
from textblob import TextBlob

# Function to get sentiment
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

# Apply sentiment analysis
data['sentiment'] = data['cleaned_text'].apply(get_sentiment)

# Display sentiment distribution
print(data['sentiment'].value_counts())

sentiment
positive    14255
negative     5724
neutral        18
Name: count, dtype: int64


In [32]:
# Calculate evaluation metrics
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.72      0.79      0.75       173
           comp.graphics       0.87      0.91      0.89       179
 comp.os.ms-windows.misc       0.93      0.88      0.91       226
comp.sys.ibm.pc.hardware       0.85      0.86      0.85       204
   comp.sys.mac.hardware       0.90      0.96      0.93       205
          comp.windows.x       0.97      0.94      0.96       186
            misc.forsale       0.91      0.78      0.84       190
               rec.autos       0.92      0.94      0.93       203
         rec.motorcycles       1.00      0.97      0.98       218
      rec.sport.baseball       0.99      0.98      0.99       192
        rec.sport.hockey       0.98      0.99      0.98       203
               sci.crypt       0.90      0.98      0.94       200
         sci.electronics       0.94      0.90      0.92       227
                 sci.med       1.00      0.95      0.98       196
         